# GAN

### 학습목표

1. Generative Model 에 대해 이해한다.
2. GAN 의 신경망 구조에 대해 이해한다.
3. GAN 의 학습방법인 Minimax Game 을 이해한다.

### 1. Generative Model

#### Generative Model

- Generative Model은 모델로부터 생성한 데이터가 실제 데이터와 비슷해지도록 학습해서 모델의 성능을 개선시킨다.

- 즉, **실제 데이터**의 <span style="color: blue">분포</span> 와 **생성한 데이터** 의 <span style="color: blue">분포</span> 가 비슷해지도록 모델을 학습시킨다. **[분포]**

- Unsupervised learning

- 단, 기존 비지도 학습 (clustering etc.) 이 데이터의 숨은 구조 (pattern) 를 파악하는 학습이라면, 

- Generative Model은 모르는 실제 데이터의 분포를 학습을 통해 단계적으로 추정 (근사) 해서 유사한 분포를 창조 

> 데이터를 학습하여 데이터 분포 $p_{model}(x)$ 를 추정하고 추정된 분포 $p_{model}(x)$ 에 따라 샘플을 생성하는 모델
<img src="./images/0_generative_model_1.png" style="width: 600px"></img>

#### Manifold Hypothesis (다양체 가설)

- 분포를 추정하기 위해 필요한 가설

- **Manifold(다양체):** 높은 차원의 공간에 정의되어 있으나, 그보다 낮은 차원으로도 잘 근사할 수 있는 연결된 점들의 집합
    - *e.g.* 만약에 data 가 3차원이라고 하더라도 2차원 평면에 근사시켰을때 그 형태가 대부분 유지되는 것
    
    
- **Manifold Hypothesis:** 고차원의 데이터들은 고차원의 공간에 고르게 퍼져있는 것이 아니라, 그보다 저차원인 다양체에 밀집되어 있음
    - 데이터가 고차원의 공간에 넓게 퍼져있으면, 그 고차원의 공간에서만 파악하기 쉬움
    
    - 데이터가 고차원 공간이라도 한군데에 몰려있으면, 오히려 그 분포를 낮은 차원으로 근사했을때 더 쉽게 파악할수 있는 경우가 존재


- ***e.g.***
    - 그림의 2차원 평면에서 연결된 점들(실루엣)이 3차원 평면이 되어도 그대로 이어져있는 것을 볼수 있음

<img src="./images/1_manifold_h_1.png" style="width: 300px"></img>

- ***e.g.***
    - 왼쪽의 롤 케익 형태 그림을 보면 데이터들은 3차원 공간에 있지만, 말린 롤을 풀어보면 오른쪽처럼 풀어진 2차원 평면에 놓여 있는 것으로 볼 수도 있음
    
    - 즉, 3차원 분포에서 뽑은 데이터가 3차원 공간 전체에 퍼져 있는 것이 아니라 실제로는 2차원 다양체에만 밀집되어 있음
    
    - 학습하기 어려운 고차원의 데이터들도 그보다 저차원의 다양체에 밀집 되어 있으므로, 생성 모델이 저차원 다양체의 분포만을 학습해도 고차원 데이터의 분포를 알수 있음

<img src="./images/2_manifold_h_2.png" style="width: 600px"></img>

#### Maximum Likelihood Estimates (최대우도법)

- 보통의 확률 추정은 주어진 변수에 데이터를 대입하여 확률을 구하는 방식

- 반대로, 최대우도법은 임의의 데이터가 주어졌을 때 해당 데이터가 추출될 확률이 가장 높은 확률분포의 변수를 찾는 것

- 즉, 데이터 $x$ 와 변수 $\theta$ 가 주어졌을 때 다음 likelihood function 을 최대화 하는 변수 $\theta$ 를 찾는 것
> $argmax_{\theta}p(x|\theta)$
>
> (신경망에서 $\theta$ 는 weight)

- 일반적으로 계산을 편하게 하기 위해, 로그를 취한 log-likelihood function 사용
> $argmax_{\theta}log(p(x|\theta))$
>
> (log 는 곱하기를 더하기로 바꿔서 값이 발산하지 않도록 scale 을 낮춰줌)

#### 처리 불가능한(Intractable) 추론

- 베이지안 방법론이 많이 쓰이는 심층 신경망에서 최대우도법으로 학습을 하는경우, 은닉층 사이의 상호작용이나 층사이의 상호작용 때문에 사후분포를 계산하기가 거의 불가능(Intractable)하다.

<img src="./images/3_generating.png" style="width: 300px"></img>
> 위 그림에서 z 는 random noise, x 는 실제와 가깝게 생성된 sample image


> $p_{\theta}(z|x) = \frac{p_{\theta}(x|z) \times p_{\theta}(z)}{p_{\theta}(x)}$
- **사후확률 (posterior probability):** $p_{\theta}(z|x)$, 실제와 비슷한 sample 이 주어졌을때 특정한 noise 가 나올 확률
- 우리가 알고싶은 것은 학습된 모델의 (실제에 근사된) 확률분포 $p_{\theta}(x)$
- 식은 간단하게 보면 bayes theorem 그대로임

> $p_{\theta}(x) = \int p_{\theta}(z) \times p_{\theta}(x|z)dz$ 
- $p_{\theta}(x)$ 를 직접 구하기 위해서는 모든 noise (z) 에 대해 적분을 계산해야함 $\rightarrow$ 거의 불가능 (intractable)

> **SOLUTION**
>
> ***변분 추론***(Variational Inference) 을 통해 **모르는** (다루기 까다로운) **확률분포** $p_{\theta}(z|x)$ 를 근방에 우리가 다루기 쉬운 **정규분포** $Q(z)$ **로 근사**
> <img src="./images/4_variational_inference.png" style="width: 400px"></img>
> (ref: [변분추론(Variational Inference)](https://ratsgo.github.io/generative%20model/2017/12/19/vi/))

#### Auto-Encoder

- GAN 이전의 Generative Model

- 변분 추론 (확률분포) 사용 X

- 입력과 비슷한 출력을 내도록 신경망을 훈련시켜, 은닉층에 입력 데이 터를 압축시켜 특징을 추출하는 기법
    - Encoder: 기존의 것에서 특징만 추출하여 압축된 형태의 벡터 (latent vector; 잠재 벡터) 를 생성
    
    - Decoder: 
        - 기존 input 을 Y label 로 사용
        
        - 기존 input 과 생성된 이미지 간의 loss 를 최소화 시키는 방식으로 학습 (back prop)
        
        - 기존 input 을 압축했다가 다시 복원시킨 output 생성


- 학습이 잘 되면, Encoder를 통해서 특징을 압축하고 Decoder를 사용 해서 다시 입력과 유사한 출력을 내는 생성 모델
    - Encoder 를 통해 압축되면서 상관성이 높은 특징만 남고 나머지는 버림
    
    - Decoder 를 거치면서 복원된 output 은 기존 input 의 주요한 특징을 그대로 유지
    
<img src="./images/5_auto_encoder.png" style="width: 600px"></img>
(ref: https://laonple.blog.me/221195944242)

#### Variational Auto-Encoder

- Auto-Encoder는 결정방식의 신경망 구조이지만 **Variational Auto Encoder(VAE)** 는 확률분포에 따른 샘플링을 수행

- VAE의 encoder는 평균벡터와 표준편차벡터 등을 학습하고 분포함수를 통해 이 값들로부터 잠재벡터 (latent vector; 압축된 특징) z를 추출
> <span style="color: blue">[특징]</span>
>
> - latent vector 를 encoder 로 직접 학습하지 않고, 
>
> - 평균 (mean vector) 과 표준편차 (standard deviation vector) 즉, **분포를 학습**시킴 (여기서 분포는 정규분포라고 가정)
>
> - 학습된 분포 $N(\mu, \sigma^{2})$ 로 부터 latent vector 를 뽑아냄 **(random sampling)**
>
> *(수식: $\mu + \sigma \times random$, random 상수를 곱함으로써 back prop 에서 미분했을때 평균이외에 분포의 다른 부분에서도 sampling 할수 있도록 함)*
>
> $\rightarrow$ latent vector 는 하나의 정해진 값이 아닌 그 **분포에 속하는 어떤 값**이라도 될수 있음
>
> $\rightarrow$ Decoder 를 통해 복원된 이미지는 기존 input 에 비해 **다양한 형태**를 가지게 됨


- VAE의 decoder는 AutoEncoder의 경우와 유사하게 z를 x로 복원하는 역할을 수행

- VAE를 최적화할 때 모든 z에 대해 고려하기 어려우므로 앞에서 설명한 변분 추론을 활용

<img src="./images/6_variational_auto_encoder.jpg" style="width: 600px"></img>
(ref: [Variational AutoEncoder](https://ratsgo.github.io/generative%20model/2018/01/27/VAE/))

#### Variational AutoEncoder와 Manifold

- 학습된 VAE로부터 생성된 결과 (데이터 다형체) 를 확인해보면, 각 축의 값 ($x = z[0]$, $y = z[1]$) 이 변함에 따라 학습된 특징또한 **연속적으로** 변화하는 것을 확인 가능
    - 여기서 $z$ 는 latent vector (2차원 이미지를 생성할 것이기 때문에 2D)


- 즉, $z$의 차원에 따라 서로 다른 변화하는 요소가 인코딩 (추출) 되었음을 볼수 있음
    - 각 축 $\rightarrow$ 하나의 추출된 feature
        - *e.g.* x 축: 웃는 정도, y 축: 머리 방향
    
<img src="./images/7_VAE_manifold_1.png" style="width: 350px"></img>
<img src="./images/8_VAE_manifold_2.png" style="width: 350px"></img>
(ref: [cs231n GAN](http://cs231n.stanford.edu/slides/2018/cs231n_2018_lecture12.pdf))

#### VAE $\leftrightarrow$ GAN

- ***VAE* (Variational AutoEncoder):** 고차원 이미지를 input 으로 사용하고 **변분추론** 방식 (확률분포) 을 통해 이미지 생성
    - 고차원 $\rightarrow$ 저차원
    
    - 데이터의 분포를 직접 근사시킴 ($log(likelihood)$ 함수를 통해 직접 확률분포를 구하는 방식)
    > input, encoder, decoder 가 하나로 연결된 형태
    >
    > *직접적인(explicit) 방식*
    

- ***GAN* (Generative Adverserial Network):** 저차원의 임의의 Random Noise를 원하는 이미지로 변형할 수 있게 **Decoder를 잘 학습**시킬수 있다면 또다른 생성 모델을 만들수 있을것이다.
    - 저차원 $\rightarrow$ 고차원 (**You know what?** Manifold Hypothesis 는 두 방향 모두 가능)
    
    - 분포를 구하는 것보다는 **생성된 결과 자체**가 더 중요
    
    - 굳이 분포함수를 직접 학습하지 **않고**, 신경망 두개를 서로 적대적으로 학습하는 과정에서 학습 데이터의 분포와 모델 출력의 분포가 일치시킴
    > generator network 를 따로두고, 저차원의 단순 random noise 로부터 결과물 이미지 샘플을 바로 생성
    >
    > *간접적인(Implicit) 방식*
    
    
<img src="./images/9_VAE2GAN.png" style="width: 300px"></img>
<img src="./images/10_GAN_conceptual.png" style="width: 500px"></img>

### 2. GAN Overview

#### GAN(Generative Adversarial Network) 개요

- Neural Network를 이용한 생성 모델의 하나

- 2014년 Ian Goodfellow가 NIPS에서 선보인 아이디어

- 2016년 NIPS conference 이후 관심이 폭발적으로 늘어나고, 많은 연구가 이루어지고 있음

- 이미지를 생성하는 것에서부터 음성, 문자에 이르기 까지 다양한 분야에 적용

#### GAN 개념

- GAN 논문 제1저자인 Ian Goodfellow은 논문 예시
> 지폐위조범(Generator)은 경찰을 최대한 열심히 속이려고 하고 
> 
> 다른 한 편에서는 경찰(Discriminator)이 위조된 지폐를 진짜와 감별하려고 (Classify)노력한다. 
>
> 이런 경쟁 속에서 두 그룹 모두 속이고 구별하는 서로 의 능력이 발전하게 되고, 
>
> 결과적으로는 진짜 지폐와 위조 지폐를 구별할 수 없을 정도(구별할 확률 $P = 0.5$, random selection)에 이른다.


- 이미지 등의 샘플을 만들어 내는 네트워크 **(Generator)** 와 만들어진 샘플을 감별하는 네트워크 **(Discriminator)** 가 있어서 대립 **(Adversarial)** 하면서 서로의 성능을 점차 개선하는 구조

- 구별할 확률이 0.5 가 되면, 학습 끝

#### GAN 구조 (1) : 첫 번째 신경망 Generator

- Generator는 Random Noise를 입력 받고 가짜 데이터를 출력한다.

- Discriminator를 속일 수 있을 정도의 가짜를 만들어내는 것이 목표다. 

- Generator는 Discriminator를 속일 수 있을 정도로 Random Noise를 학습 데이터와 유사한 패턴으로 변환하는 함수를 학습한다.

<img src="./images/11_generator.png" style="width: 600px"></img>
> 여기서 $z$ 는 random noise, 생성된 sample 은 $p_{\theta}(x|z) \rightarrow G(z)$

#### GAN 구조 (2) : 두 번째 신경망 Discriminator

- Discriminator는 실제 학습 데이터와 Generator를 통해 만들어진 가짜 데이터를 입력받고, 입력된 데이터가 학습 데이터에 포함된 진짜 데이터일 확률을 출력

- 판별의 성공확률을 최대한 높이는 것이 목적이기 때문에, 정분류율을 높여야 하고 오분류율은 줄이는 방향으로 학습

<img src="./images/12_discriminator.png" style="width: 570px"></img>
> real world sample $x$ 는 항상 1 (real) 로, generated sample $G(z)$ 는 항상 0 (fake) 로 판별하는 것이 목적

#### GAN 구조 (3) : 데이터의 확률분포

- Dataset을 구성하는 데이터들은 일반적으로 어떤 분포를 따름

- 만약 $G$가 해당 데이터의 확률분포를 알아내서 오른쪽 그림처럼 정확히 모사할 수 있다면? $G$가 생성한 샘플은 실제 데이터와 구별 할 수 없을 것임

- Discriminator 가 구별 **가능**한 경우

> <img src="./images/13_prob_dist_1.png" style="width: 250px"></img>
>
> generator 로부터 생성된 데이터의 분포 $G(z) \rightarrow N(\mu_{1}, \sigma^{2})$, training sample 의 분포 $x \rightarrow N(\mu_{2}, \sigma^{2})$
>
> 두 분포의 위치가 아직 차이남

- Discriminator 가 구별 **불가능**한 경우

> <img src="./images/14_prob_dist_2.png" style="width: 200px"></img>
>
> generator 로부터 생성된 데이터의 분포 $G(z)$ $==$ training sample 의 분포 $x \rightarrow N(\mu_{\gamma}, \sigma^{2})$
>
> 두 분포의 위치가 완전히 겹쳐서 전혀 차이나지 않음


#### GAN 구조 (4) : 학습 방향

- ***Generative 모델 $(G)$:*** 입력된 Random Noise를 우리가 갖고있는 데이터 **$x$의 확률분포로 변환**하는 함수를 학습

- ***Discriminator model $(D)$:*** 현재 자기가 보고있는 샘플이 training data에서 온 진짜 data인지 혹은 $G$로부터 만들어진 것인지를 구별하여 각각의 경우에 대한 확률을 계산하고, 진위여부를 판별

#### GAN 구조 (5) : Cost Function

- 하나의 cost function 만 관리하면 되는 기존의 neural net 들과는 다르게 Generator 와 Discriminator, 총 두개의 loss function 존재

- 학습이 Discriminator 와 Generator 사이를 번갈아가면서 진행됨 (Alternate)

<img src="./images/15_two_loss_function.png" style="width: 600px"></img>

#### GAN 구조 (6) : 적대적 학습

**[1: fix Generator weight]**

- 맨 처음 학습할때 Generator fake 이미지 생성 

- Discriminator 가 아직 학습되지 못한 (진위를 제대로 구별하지 못하는) 상황이므로 Discriminator 를 먼저 학습시킴

- 이때 Generator의 가중치가 변하지 않도록 고정하여, Discriminator 의 학습 (backprop; weight update) 의 **영향을 받지 않도록** 해야함

<img src="./images/16_GAN_train_1.png" style="width: 600px"></img>

**[2: update ONLY Discriminator]**

- Generator에서 샘플을 생성하고, 생성된 샘플 $G(z)$ 을 실제 샘플 $x$ 과 함께 Discriminator를 통과시킴 ($z$: random noise)

- Back Propagation 을 통해 Discriminator의 가중치를 갱신하여 학습

<img src="./images/17_GAN_train_2.png" style="width: 600px"></img>

**[3: update ONLY Generator]**

- 이번에는 Generator를 학습시키기 위해 Discriminator의 가중치를 고정

- Generator가 샘플을 생성 $G(z)$ 하고 Discriminator를 통과시킴 (Generator를 학습시킬 때에는 실제 샘플 $x$ 은 사용하지 않음)

- 마찬가지로 오류 역전파를 통해 Generator의 가중치를 갱신하여 학습

- 가중치 값들이 좀더 실제에 가까운 분포를 형성하는 방향으로 update 됨

<img src="./images/18_GAN_train_3.png" style="width: 600px"></img>

### 3. Minimax Game

#### Minimax Game 개념

- GAN은 원하는 패턴을 생성하는 문제를 Minimax Game 이론으로 접근함

- Minimax 이론은 발생할수있는 손실 중 가장 최대(Max)의 손실을 최소화(Min)하는 의사결정 원칙

- 즉, 발생 가능한 최악의 손실(Loss)을 최소로 줄이는 방법

(ref: [Minimax algorithm explained](https://www.youtube.com/watch?v=l-hh51ncgDI))

#### Minimax Game in GAN

<img src="./images/19_minimax_1.png" style="width: 400px"></img>
(ref: [GAN blogspot](http://dl-ai.blogspot.com/))

- 패턴을 출력하는 GAN은 새로운 이미지 패턴을 잘 생성해내는게 목적

- 따라서 **Generator**의 입장에서 Minimax Game을 적용

- Generator의 입장에서 가장 최악(Max)의 경우는 Discriminator가 주어진 임의의 데이터를 완벽히 판별하는 경우

- 따라서 Discriminator의 판별 정확도(Loss)를 최소화(Min)하는 방향으로 학습을 진행
    - 위폐를 최대한 잘만들면 가능


- Discriminator의 판별 정확도 중 최솟값은 0.5 (O와 X를 구분할수 없어 무작위로 고를때의 확률 0.5)

- 따라서 학습이 진행될수록 Discriminator가 주어진 데이터를 판별할 확률은 **0.5로 수렴**해야함

#### Training GAN

<img src="./images/21_minimax_train.png" style="width: 800px"></img>
> - 검은 점선: 학습 데이터 (real-world training data) 분포 $x$ 
>
> - 녹색 실선: Generator $G$ 에서 생성되는 데이터의 분포 $G(z)$ (input:  random noise $z$)
>
> - 파란 점선: Discriminator $D$ 의 출력값 $\rightarrow$ 1 (real) 또는 0 (fake) 
>    - 마치 sigmoid activation function 의 모습을 하고있음

**(a)** Generator 에서 random noise 를 받아서 첫 이미지 생성
- 아직 Generator 가 학습되지 않았으므로 실제 데이터에 비해 많이 치우친 분포를 가진 데이터를 생성

- Discriminator 또한 아직 학습되지 않았으므로 출력을 결정함에 있어서 불확실 (출력이 흔들리고 부분적으로만 일치)

**(b)** Discriminator 학습
- Discriminator 가 이제서야 확실한 결론을 낼수 있게됨

- 이때 Generator 의 weight 는 고정되어 있어야 함

**(c)** Generator 학습
- G의 분포 와 학습 데이터의 분포가 점점더 근접 

**(d)** $D$ 와 $G$ 반복학습후 수렴 $\rightarrow$ 학습 종료
- 실제 데이터 분포 $x$ 와 생성된 데이터 분포 $G(z)$ 가 정확히 일치하면서 Discriminator 의 출력값이 평형 상태인 0.5에 도달한다. (더이상 구별 불가)

---

### References

1. ["Deep Learning" by Ian Goodfellow, Yoshua Bengio, Aaron Couville, The MIT Press (Web Book)](http://www.deeplearningbook.org/)

2. [NIPS 2016 Tutorial: Generative Adversarial Networks Ian Goodfellow](https://arxiv.org/pdf/1701.00160.pdf)

3. [GAN 개요 - 라온피플 머신러닝 아카데미](https://laonple.blog.me/221195944242)

4. [차원축소 PCA](http://excelsior-cjh.tistory.com/167)